In [1]:
import numpy as np
import sklearn as skl
from numpy import genfromtxt, matlib
from sklearn import kernel_ridge
import os
from sklearn.model_selection import GridSearchCV
print(len(os.sched_getaffinity(0)))

16


In [2]:
path_to_train = '/home/sathvikc/AMATH-482-2/AMATH-482/Homework Assignments/Homework 3/wine_training.csv'
path_to_test = '/home/sathvikc/AMATH-482-2/AMATH-482/Homework Assignments/Homework 3/wine_test.csv'

train = genfromtxt(path_to_train, delimiter=',')
test = genfromtxt(path_to_test, delimiter=',')

print('TRAIN SET')
print(train)
print('ALL BUT LAST')
print(train[:, :-1])
print('LAST')
print(train[:, -1])

TRAIN SET
[[ 6.1   0.6   0.08 ...  0.54 11.    5.  ]
 [10.3   0.53  0.48 ...  0.59  9.3   6.  ]
 [ 6.9   0.49  0.19 ...  0.64  9.8   6.  ]
 ...
 [ 7.    0.38  0.49 ...  0.77 11.4   6.  ]
 [10.    0.35  0.47 ...  0.52 12.    6.  ]
 [ 7.    0.6   0.3  ...  1.17 10.2   5.  ]]
ALL BUT LAST
[[ 6.1   0.6   0.08 ...  3.38  0.54 11.  ]
 [10.3   0.53  0.48 ...  3.12  0.59  9.3 ]
 [ 6.9   0.49  0.19 ...  3.38  0.64  9.8 ]
 ...
 [ 7.    0.38  0.49 ...  3.39  0.77 11.4 ]
 [10.    0.35  0.47 ...  3.23  0.52 12.  ]
 [ 7.    0.6   0.3  ...  3.3   1.17 10.2 ]]
LAST
[5. 6. 6. ... 6. 6. 5.]


In [3]:
X_train_N = train[:, :-1].shape[0]

X_train_mean = np.mean(train[:, :-1], axis=0)
X_train_std = np.std(train[:, :-1], axis=0)
print(X_train_mean)
print(X_train_std)
X_train_normal = (train[:, :-1] - matlib.repmat(X_train_mean, X_train_N, 1))/matlib.repmat(X_train_std, X_train_N, 1)

Y_train_N = train[:, -1].shape[0]

Y_train_mean = np.mean(train[:, -1], axis=0)
Y_train_std = np.std(train[:, -1], axis=0)
print(Y_train_mean)
print(Y_train_std)

Y_train_normal = (train[:, -1] - Y_train_mean)/Y_train_std

[ 8.35183857  0.52507623  0.27384753  2.55399103  0.08926547 16.15695067
 46.38699552  0.99683448  3.31076233  0.66102242 10.39699552]
[1.73802726e+00 1.78924336e-01 1.95148774e-01 1.42389434e+00
 5.11090930e-02 1.06931743e+01 3.14087940e+01 1.87298299e-03
 1.52519210e-01 1.78183748e-01 1.04853788e+00]
5.638565022421525
0.7987331359432255


In [4]:
X_test_N = test[:, :-1].shape[0]

X_test_mean = np.mean(test[:, :-1], axis=0)
X_test_std = np.std(test[:, :-1], axis=0)

X_test_normal = (test[:, :-1] - matlib.repmat(X_test_mean, X_test_N, 1))/matlib.repmat(X_test_std, X_test_N, 1)

Y_test_N = test[:, -1].shape[0]

Y_test_mean = np.mean(test[:, -1], axis=0)
Y_test_std = np.std(test[:, -1], axis=0)

Y_test_normal = (test[:, -1] - Y_test_mean)/Y_test_std

In [5]:
#Use linear regression to fit a linear model to the training set 
LinearRegressor = skl.linear_model.LinearRegression()
LinearRegressor.fit(X_train_normal, Y_train_normal)
predicted_train = LinearRegressor.predict(X_train_normal)
print(f'LINEAR REGRESSION TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, Y_train_normal)}')
predicted_test = LinearRegressor.predict(X_test_normal)
print(f'LINEAR REGRESSION TEST MSE: {skl.metrics.mean_squared_error(predicted_test, Y_test_normal)}')

LINEAR REGRESSION TRAINING MSE: 0.6278484956554882
LINEAR REGRESSION TEST MSE: 0.7021527395264071


In [6]:
#Use kernel ridge regression to fit a non-linear model to the training set

#Gaussian (RBF) Kernel 
RBFRegresor = kernel_ridge.KernelRidge(kernel = 'rbf')  

#Laplacian kernel
LaplacianRegresor = kernel_ridge.KernelRidge(kernel = 'laplacian')

In [7]:
# # range of values of sigma to try 
# K_sgm = 10
# K_lmbd = 10

# sgm = np.linspace(1, 3, K_sgm)
# lmbd = np.linspace(-3, -1, K_lmbd)

# #RBF 
# cv = GridSearchCV(RBFRegresor, param_grid = {
#     'alpha': 2 ** lmbd,
#     'gamma': 1 / (2*(2**sgm)**2)
#     }, cv = 10, n_jobs = -1)

# cv.fit(X_train_normal, Y_train_normal)
# print(cv.best_params_)

In [8]:
# best_alpha_rbf = cv.best_params_['alpha']
# best_gamma_rbf = cv.best_params_['gamma']
# print(best_alpha_rbf)
# print(best_gamma_rbf)

In [9]:
# range of values of sigma to try 


K_sgm = 10
K_lmbd = 10

sgm = np.linspace(1, 3, K_sgm)
lmbd = np.linspace(-3, -1, K_lmbd)

scores = np.zeros((K_sgm, K_lmbd))
scores_std = np.zeros((K_sgm, K_lmbd))

KRR_CV = skl.kernel_ridge.KernelRidge(kernel='rbf')

for i in range(K_sgm):

  KRR_CV.gamma = 1/(2*(2**sgm[i])**2)

  for j in range(K_lmbd): 

    KRR_CV.alpha = (2**lmbd[j])
    this_score = skl.model_selection.cross_val_score(KRR_CV, X_train_normal, Y_train_normal, scoring= 'neg_mean_squared_error', cv = 10)

    scores[i,j] = (np.mean(this_score))
    scores_std[i,j] = (np.std(this_score))

print(scores)

ij_max = np.array( np.where( scores == scores.max() ), dtype=int).flatten()

print(ij_max)

print(scores.max())

print('log_2 sg:', sgm[ij_max[0]], 'log_2_lmbd: ', lmbd[ij_max[1]])
print('alpha: ', 2**lmbd[ij_max[1]])
print('gamma: ', 1/(2*(2**sgm[ij_max[0]])**2))
best_alpha_rbf = 2**lmbd[ij_max[1]]
best_gamma_rbf = 1/(2*(2**sgm[ij_max[0]])**2)

[[-0.62947824 -0.62339646 -0.61808084 -0.61348839 -0.60957767 -0.60630862
  -0.60364265 -0.60154271 -0.59997359 -0.59890221]
 [-0.6163228  -0.61207054 -0.60843803 -0.60537789 -0.60284541 -0.60079868
  -0.59919871 -0.59800974 -0.59719946 -0.59673935]
 [-0.60621485 -0.60364513 -0.60151692 -0.5997898  -0.59842706 -0.59739575
  -0.59666669 -0.59621446 -0.59601724 -0.59605663]
 [-0.59990619 -0.59856804 -0.59752259 -0.59674482 -0.5962126  -0.59590639
  -0.59580885 -0.59590447 -0.59617928 -0.5966206 ]
 [-0.59660205 -0.59609636 -0.59581071 -0.59572773 -0.59583057 -0.59610274
  -0.59652802 -0.59709045 -0.59777443 -0.59856499]
 [-0.59581242 -0.59605441 -0.59645918 -0.59700548 -0.5976725  -0.5984399
  -0.59928805 -0.60019845 -0.60115425 -0.60214093]
 [-0.59846861 -0.59928327 -0.60016743 -0.60109981 -0.60206049 -0.60303142
  -0.60399713 -0.60494556 -0.6058688  -0.60676383]
 [-0.604041   -0.60496812 -0.60587503 -0.60674814 -0.60757747 -0.60835743
  -0.60908729 -0.60977148 -0.61041953 -0.61104571]
 

In [10]:
# # range of values of sigma to try 
# K_sgm = 10
# K_lmbd = 10

# sgm = np.linspace(-4, 4, K_sgm)
# lmbd = np.linspace(-4, 4, K_lmbd)

# #Laplacian
# cv = GridSearchCV(LaplacianRegresor, param_grid = {
#     'alpha': 2 ** lmbd,
#     'gamma': 1 / (2 ** (sgm))
#     }, cv = 5, n_jobs=-1)

# cv.fit(X_train_normal, Y_train_normal)

In [11]:
# best_alpha_laplacian = cv.best_params_['alpha']
# best_gamma_laplacian = cv.best_params_['gamma']
# print(best_alpha_laplacian)
# print(best_gamma_laplacian)

In [12]:
# range of values of sigma to try 


K_sgm = 10
K_lmbd = 10

sgm = np.linspace(0, 4, K_sgm)
lmbd = np.linspace(-3, 1, K_lmbd)

scores = np.zeros((K_sgm, K_lmbd))
scores_std = np.zeros((K_sgm, K_lmbd))

KRR_CV = skl.kernel_ridge.KernelRidge(kernel = 'laplacian')

for i in range(K_sgm):

  KRR_CV.gamma = 1 / (2 ** (sgm[i]))

  for j in range(K_lmbd): 

    KRR_CV.alpha = (2**lmbd[j])
    this_score = skl.model_selection.cross_val_score(KRR_CV, X_train_normal, Y_train_normal, scoring= 'neg_mean_squared_error', cv = 10)

    scores[i,j] = (np.mean(this_score))
    scores_std[i,j] = (np.std(this_score))

print(scores)

ij_max = np.array( np.where( scores == scores.max() ), dtype=int).flatten()

print(ij_max)

print(scores.max())

print('log_2 sg:', sgm[ij_max[0]], 'log_2_lmbd: ', lmbd[ij_max[1]])
print('alpha: ', 2**lmbd[ij_max[1]])
print('gamma: ', 1/(2**sgm[ij_max[0]]))
best_alpha_laplacian = 2**lmbd[ij_max[1]]
best_gamma_laplacian = 1/(2*(2**sgm[ij_max[0]])**2)

[[-0.7051611  -0.70887657 -0.71400115 -0.72098566 -0.7303263  -0.74248783
  -0.75778525 -0.77624987 -0.79752907 -0.82087101]
 [-0.62705943 -0.63007216 -0.63433652 -0.64029752 -0.64846855 -0.65937474
  -0.67346362 -0.69100216 -0.71199056 -0.7361167 ]
 [-0.57823357 -0.58017514 -0.58311814 -0.58746162 -0.59366238 -0.60218047
  -0.61340464 -0.62758299 -0.6447884  -0.66493113]
 [-0.55382838 -0.55462268 -0.55624143 -0.55906579 -0.56351447 -0.56998357
  -0.57877778 -0.59005886 -0.60383491 -0.61999761]
 [-0.54626023 -0.54582335 -0.54614391 -0.54760178 -0.55057875 -0.55539831
  -0.56227295 -0.57127798 -0.58236275 -0.59539654]
 [-0.54790922 -0.54625624 -0.5454519  -0.54586723 -0.54782418 -0.55154641
  -0.55713116 -0.56455155 -0.57369038 -0.58439614]
 [-0.55284155 -0.55018134 -0.54861865 -0.5484777  -0.54998254 -0.55322856
  -0.55818084 -0.56470231 -0.57260697 -0.58172462]
 [-0.55773872 -0.55456078 -0.55281511 -0.55271988 -0.55435794 -0.55767403
  -0.56249982 -0.56860448 -0.57576081 -0.58381099]


In [13]:
#MSE of all 3 models

#Linear Regression
print(f'LINEAR REGRESSION TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, Y_train_normal)}')
print(f'LINEAR REGRESSION TEST MSE: {skl.metrics.mean_squared_error(predicted_test, Y_test_normal)}')

#RBF
RBFRegresor = kernel_ridge.KernelRidge(alpha = best_alpha_rbf, kernel = 'rbf', gamma = best_gamma_rbf)
RBFRegresor.fit(X_train_normal, Y_train_normal)
predicted_train = RBFRegresor.predict(X_train_normal)
print(f'GAUSSIAN KERNEL TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, Y_train_normal)}')
predicted_test = RBFRegresor.predict(X_test_normal)
print(f'GAUSSIAN KERNEL TEST MSE: {skl.metrics.mean_squared_error(predicted_test, Y_test_normal)}')

#Laplacian
LaplacianRegresor = kernel_ridge.KernelRidge(alpha = best_alpha_laplacian, kernel = 'laplacian', gamma = best_gamma_laplacian)
LaplacianRegresor.fit(X_train_normal, Y_train_normal)
predicted_train = LaplacianRegresor.predict(X_train_normal)
print(f'LAPLACIAN KERNEL TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, Y_train_normal)}')
predicted_test = LaplacianRegresor.predict(X_test_normal)
print(f'LAPLACIAN KERNEL TEST MSE: {skl.metrics.mean_squared_error(predicted_test, Y_test_normal)}')

LINEAR REGRESSION TRAINING MSE: 0.6278484956554882
LINEAR REGRESSION TEST MSE: 0.7021527395264071
GAUSSIAN KERNEL TRAINING MSE: 0.45487888889595507
GAUSSIAN KERNEL TEST MSE: 0.642230584877313
LAPLACIAN KERNEL TRAINING MSE: 0.39264462112490023
LAPLACIAN KERNEL TEST MSE: 0.6297267121155207


In [14]:
#Predict on new batch of wines 
path_to_new_wine = '/home/sathvikc/AMATH-482-2/AMATH-482/Homework Assignments/Homework 3/wine_new_batch.csv'
new_wine = genfromtxt(path_to_new_wine, delimiter=',')

X_new_wine_N = new_wine.shape[0]

X_new_wine_mean = np.mean(new_wine, axis=0)
X_new_wine_std = np.std(new_wine, axis=0)

X_new_wine_normal = (new_wine - matlib.repmat(X_new_wine_mean, X_new_wine_N, 1))/matlib.repmat(X_new_wine_std, X_new_wine_N, 1)

In [16]:
#Linear Regression 
linear_predict = LinearRegressor.predict(X_new_wine_normal)

#RBF
rbf_predict = RBFRegresor.predict(X_new_wine_normal)

#Laplacian
laplacian_predict = LaplacianRegresor.predict(X_new_wine_normal)

#De-normalize the predictions
linear_predict = (linear_predict * Y_train_std) + Y_train_mean
rbf_predict = (rbf_predict * Y_train_std) + Y_train_mean
laplacian_predict = (laplacian_predict * Y_train_std) + Y_train_mean

#print the predictions 
print(f'LINEAR PREDICTION: {linear_predict}')
print(f'RBF PREDICTION: {rbf_predict}')
print(f'LAPLACIAN PREDICTION: {laplacian_predict}')

LINEAR PREDICTION: [6.15311173 4.77584118 5.67006291 5.73158471 5.86222458]
RBF PREDICTION: [6.44841878 4.81465157 5.31860301 5.23211189 5.1219432 ]
LAPLACIAN PREDICTION: [6.2243997  4.86364547 5.50656253 5.4880713  5.54749558]
